In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp

from IPython.display import *

# **Question 1**


## <a href="https://dailytrojan.com/2023/08/07/bonds/">Optimal Bond Allocation for the USC Investment Office</a>
<br>
<font size="+1">
    <ul>
        <li>Assume the <a href="http://departmentsdirectory.usc.edu/investment_office.html">USC Investment Office</a> has decided they want to allocate $100,000 of the endowment fund into two different bonds: a corporate bond, and a government bond. The bonds have the following</li>
        <br>
        <ul>
        <li><a href="https://www.investopedia.com/terms/b/bond-yield.asp">yield</a> (coupon relative to current price),</li>
        <br>
        <li>risk level (some KPI statistic that measures risk),</li>
        <br>
        <li>and <a href="https://en.wikipedia.org/wiki/Bond_(finance)#Maturity">maturity</a> (time before your principal is returned).</li>
        <br>
        </ul>
        <li><table>
  <tr>
    <th>Bond</th>
    <th>Yield</th>
    <th>Risk Level</th>
    <th>Maturity</th>
  </tr>
  <tr>
    <td>Corporate</td>
    <td>$4\%$</td>
    <td>2</td>
    <td>3 years</td>
  </tr>
  <tr>
    <td>Government</td>
    <td>$3\%$</td>
    <td>1</td>
    <td>4 years</td>
  </tr>
</table></li>
        <br>
        <li>One of the endowment's investment officer would like allocate the $\$100,000$ slice of the endowment so that</li>
        <br>
        <ul>
            <li>the <b>average</b> risk level of the portfolio is at most $1.5$,</li>
            <br>
            <li>the <b>average</b> maturity is at most $3.6$ years,</li>
            <br>
            <li>and any amount not invested in the bonds will be kept in a cash account that is assumed to generate no interest and does not contribute to the average risk level; that is, assume cash has zero yield, zero risk level, and zero maturity.</li>
            <br>
</font>

In [13]:
# The decision variables are the two different bond that the USC Investment Office wants to allocate their $100,000 endowment fund

allocations = cp.Variable(2)

In [14]:
# The objective is to maximize the expected yield, which is written in this linear equation
expected_yield = 0.04*allocations[0] + 0.03*allocations[1]
objective = cp.Maximize(expected_yield)

In [15]:
# We set the constraints to match the yield and risk level according to the maturity.
# Remember, since the endowment fund is only $100,000 then the two allocations could not exceed the amount of the original fund.
constraints = [(allocations[0] + allocations[1] <= 100_000),
                (2*allocations[0] + 1*allocations[1] <= 150_000),
               (3*allocations[0] + 4*allocations[1] <= 360_000),
                (allocations >= 0)]

In [16]:
# Print the problem and the solver
problem = cp.Problem(objective, constraints)
print(problem)

maximize 0.04 @ var91[0] + 0.03 @ var91[1]
subject to var91[0] + var91[1] <= 100000.0
           2.0 @ var91[0] + 1.0 @ var91[1] <= 150000.0
           3.0 @ var91[0] + 4.0 @ var91[1] <= 360000.0
           0.0 <= var91


In [17]:
problem.solve()

3499.999999925091

In [18]:
problem.solve(verbose=True)

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Jan 16 06:34:26 PM: Your problem has 2 variables, 4 constraints, and 0 parameters.
(CVXPY) Jan 16 06:34:26 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 16 06:34:26 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 16 06:34:26 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 16 06:34:26 PM: Using cached ASA map, for faster compilation (bypassing reduction chain).
(CVXPY) Jan 16 06:34:26 PM: Finished problem compilation (took 

3499.999999925091

In [19]:
#This code shows much money needs to be allocated for each bond
allocations.value

array([50000.00000268, 49999.99999393])

In [20]:
display(Markdown("<b>The solution to this problem would be to invest 0 in cash, 50000 in corporate bond and 50000 in government bond.</b>"))

<b>The solution to this problem would be to invest 0 in cash, 50000 in corporate bond and 50000 in government bond.</b>

In [21]:
np.sum(allocations.value)

99999.99999660859

In [22]:
round(np.sum(allocations.value))

100000

# **Question 2**

## Bond Dedication
   <li>A common application of optimization for decision making within a company is to cover <b>known</b> liabilities (unknown liabilities are a different story) by constructing a <b>dedicated</b> fixed-income (bond) portfolio.</li>
    <br>
    <li>A <b>dedicated</b> bond portfolio is a portfolio of bonds constructed today with cash flows that offset the liabilities.</li>
    <br>
    <li>Often, when the liabilities span multiple years, many companies make the simplifying assumption that the only sources of risk are changes in interest rates (more specifically, the term structure of interest rates).</li></font>
    <br>
    <li>Consider a business facing the general problem of funding a stream of liabilities, such as labor costs, or the planning of an acquisition, expansion, product development, or other operational costs, that extends over the future.</li>
    <br>
    <li>Assume you have a forecast of future liabilities, which is reasonably accurate (not so easy to do in practice).</li>
    <br>
    <li>Specifically, suppose a pension fund needs to cover some liabilities, due to people retiring, in the next six years.</li>
    <br>
    <li>The cash requirements (in millions) needing to be funded are:<table>
  <tr>
    <th>Year</th>
    <th>1</th>
    <th>2</th>
    <th>3</th>
      <th>4</th>
      <th>5</th>
      <th>6</th>
  </tr>
  <tr>
    <td>Liability Needing To Be Covered</td>
    <td>100</td>
    <td>200</td>
    <td>800</td>
      <td>100</td>
      <td>800</td>
      <td>1200</td>
  </tr>
            </table></li>
        <br>
        <li>Assume the pension fund can invest in the following ten government bonds of various maturities with the given cash flows and current prices (both in millions): <table>
  <tr>
    <th>Bond $\downarrow$,  Year $\rightarrow$</th>
    <th>1</th>
    <th>2</th>
    <th>3</th>
      <th>4</th>
      <th>5</th>
      <th>6</th>
      <th>Price</th>
  </tr>
  <tr>
    <td>Bond 1</td>
    <td>10</td>
    <td>10</td>
    <td>10</td>
      <td>10</td>
      <td>10</td>
      <td>100+10</td>
      <td>109</td>
  </tr>
              <tr>
    <td>Bond 2</td>
    <td>7</td>
    <td>7</td>
    <td>7</td>
      <td>7</td>
      <td>7</td>
      <td>100+7</td>
      <td>94.8</td>
  </tr>
              <tr>
    <td>Bond 3</td>
    <td>8</td>
    <td>8</td>
    <td>8</td>
      <td>8</td>
      <td>8</td>
      <td>100+8</td>
      <td>99.5</td>
  </tr>
              <tr>
    <td>Bond 4</td>
    <td>6</td>
    <td>6</td>
    <td>6</td>
      <td>6</td>
      <td>100+6</td>
      <td>n/a</td>
      <td>93.1</td>
  </tr>
              <tr>
    <td>Bond 5</td>
    <td>7</td>
    <td>7</td>
    <td>7</td>
      <td>7</td>
      <td>100+7</td>
      <td>n/a</td>
      <td>97.2</td>
  </tr>
              <tr>
    <td>Bond 6</td>
    <td>5</td>
    <td>5</td>
    <td>5</td>
      <td>100+5</td>
      <td>n/a</td>
      <td>n/a</td>
      <td>92.7</td>
  </tr>
              <tr>
    <td>Bond 7</td>
    <td>10</td>
    <td>10</td>
    <td>100+10</td>
      <td>n/a</td>
      <td>n/a</td>
      <td>n/a</td>
      <td>110</td>
  </tr>
              <tr>
    <td>Bond 8</td>
    <td>8</td>
    <td>8</td>
    <td>100+8</td>
      <td>n/a</td>
      <td>n/a</td>
      <td>n/a</td>
      <td>104</td>
  </tr>
              <tr>
    <td>Bond 9</td>
    <td>7</td>
    <td>100+7</td>
    <td>n/a</td>
      <td>n/a</td>
      <td>n/a</td>
      <td>n/a</td>
      <td>102</td>
  </tr>
              <tr>
    <td>Bond 10</td>
    <td>100</td>
    <td>n/a</td>
    <td>n/a</td>
      <td>n/a</td>
      <td>n/a</td>
      <td>n/a</td>
      <td>95.2</td>
  </tr>
            </table>
        </li>
        <br>
        <li>Find the least expensive portfolio of bonds whose cash flows will be sufficient to cover the cash requirements. This portfolio of bonds is the <b>dedicated portfolio</b>.</li>
        <br>
        <li>After every year, the dedicated bond portfolio must cover the liability for that year, but it can also pay more than the liability, which is called a surplus.</li>
        <br>
        <li>The yearly surplus is something the pension fund can choose to control.</li>
        <br>
        <li>Assume any surplus cash can be carried from one year to the next, but will earn zero interest.</li>
        <br>
        

In [ ]:
# Create the decision variables for this optimization problem.
# Since the pension fund could be invested into ten government bonds, the investment for each bond is the decision variable

bond_investment = cp.Variable(10)

bond_1 = bond_investment[0]
bond_2 = bond_investment[1]
bond_3 = bond_investment[2]
bond_4 = bond_investment[3]
bond_5 = bond_investment[4]
bond_6 = bond_investment[5]
bond_7 = bond_investment[6]
bond_8 = bond_investment[7]
bond_9 = bond_investment[8]
bond_10 = bond_investment[9]

In [ ]:
# After every year, the dedicated bond portfolio must cover the liability for that year, but it can pay more than the liability (called surplus)
# The surplus can be controlled meaning it would be another decision variable
cash_surplus = cp.Variable(6)

surplus_1 = cash_surplus[0]
surplus_2 = cash_surplus[1]
surplus_3 = cash_surplus[2]
surplus_4 = cash_surplus[3]
surplus_5 = cash_surplus[4]
surplus_6 = cash_surplus[5]

In [ ]:
#Create a linear equation describing the bond The objective of this solver is to minimize the price
price = 109*bond_1 + 94.8*bond_2 + 99.5*bond_3 + 93.1*bond_4 + 97.2*bond_5 + 92.7*bond_6 + 110*bond_7 + 104*bond_8 + 102*bond_9 + 95.2*bond_10
objective = cp.Minimize(price)

In [ ]:
# Write down the constraints which includes the liability and surplus.
# Since the surplus and long term-investment could never be nominally 0, that constraint is also added.
year_one_constraint = (10 * bond_1 + 7 * bond_2 + 8 * bond_3 + 6 * bond_4 + 7 * bond_5 + 5 * bond_6 + 10 * bond_7 + 8 * bond_8 + 7 * bond_9 + 100 * bond_10 == 100 + surplus_1)
year_two_constraint = (10 * bond_1 + 7 * bond_2 + 8 * bond_3 + 6 * bond_4 + 7 * bond_5 + 5 * bond_6 + 10 * bond_7 + 8 * bond_8 + 107 * bond_9 + surplus_1 == 200 + surplus_2)
year_three_constraint = (10 * bond_1 + 7 * bond_2 + 8 * bond_3 + 6 * bond_4 + 7 * bond_5 + 5 * bond_6 + 110 * bond_7 + 108 * bond_8 + surplus_2 == 800 + surplus_3)
year_four_constraint = (10 * bond_1 + 7 * bond_2 + 8 * bond_3 + 6 * bond_4 + 7 * bond_5 + 105 * bond_6 + surplus_3 == 100 + surplus_4)
year_five_constraint = (10 * bond_1 + 7 * bond_2 + 8 * bond_3 + 106 * bond_4 + 107 * bond_5 + surplus_4 == 800 + surplus_5)
year_six_constraint = (110 * bond_1 + 107 * bond_2 + 108 * bond_3 + surplus_5 == 1200 + surplus_6)
long_only_investment = (bond_investment >= 0)
surplus_only = (cash_surplus >= 0)

In [ ]:
problem = cp.Problem(objective, [year_one_constraint,
                              year_two_constraint,
                              year_three_constraint,
                              year_four_constraint,
                              year_five_constraint,
                              year_six_constraint,
                              long_only_investment,
                              surplus_only])

In [ ]:
problem.solve()

2305.69164800379

In [ ]:
problem.solve(verbose=True)

                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Jan 16 10:06:53 AM: Your problem has 16 variables, 8 constraints, and 0 parameters.
(CVXPY) Jan 16 10:06:53 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 16 10:06:53 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 16 10:06:53 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 16 10:06:53 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 16 10:06:53 AM: Using cached ASA map, for faster compilation

2305.69164800379

In [ ]:
bond_investment.value

array([-0.        , 11.21495327, -0.        ,  6.63384513, -0.        ,
       -0.        , -0.        ,  6.00868362, -0.        , -0.        ])

In [ ]:
bond_inv = np.round(bond_investment.value, 2)
bond_inv

array([-0.  , 11.21, -0.  ,  6.63, -0.  , -0.  , -0.  ,  6.01, -0.  ,
       -0.  ])

In [ ]:
# This is the least expensive portfolio of bonds whose cash flow will be sufficient to cover the cash requirements.
bond_no = 1

for i in np.round(bond_investment.value, 2):
  print("Buy", i, "units in bond number", bond_no)
  bond_no += 1

Buy -0.0 units in bond number 1
Buy 11.21 units in bond number 2
Buy -0.0 units in bond number 3
Buy 6.63 units in bond number 4
Buy -0.0 units in bond number 5
Buy -0.0 units in bond number 6
Buy -0.0 units in bond number 7
Buy 6.01 units in bond number 8
Buy -0.0 units in bond number 9
Buy -0.0 units in bond number 10


In [ ]:
# This is the amount of surplus for each year
surplus = np.round(cash_surplus.value, 2)
surplus

year_number = 1
for i in surplus:
    print("The cash surplus value is $", i,"million in year", year_number)
    year_number +=1


The cash surplus value is $ 66.38 million in year 1
The cash surplus value is $ 32.75 million in year 2
The cash surplus value is $ -0.0 million in year 3
The cash surplus value is $ 18.31 million in year 4
The cash surplus value is $ -0.0 million in year 5
The cash surplus value is $ -0.0 million in year 6


# **Question 3**

## Quadratic Asset Allocation
<br>
<font size="+1">
    <ul>
        <li>Assume you have estimated, with the best possible machine learning algorithm, the one-year correlations and standard deviations of the following groups of large stocks, small stocks, and bonds.</li>
        <br>
        <li><table>
  <tr>
    <th bgcolor="dodgerblue">Correlations</th>
    <th>Large Stocks</th>
    <th>Small Stocks</th>
    <th>Bonds</th>
      <th bgcolor="pink">Standard Deviation</th>
  </tr>
  <tr>
    <td>Large Stocks</td>
    <td bgcolor="dodgerblue" >1</td>
    <td bgcolor="dodgerblue">0.6</td>
    <td bgcolor="dodgerblue">0.2</td>
      <td bgcolor="pink">0.12</td>
  </tr>
  <tr>
    <td>Small Stocks</td>
    <td bgcolor="dodgerblue">0.6</td>
    <td bgcolor="dodgerblue">1</td>
    <td bgcolor="dodgerblue">0.5</td>
      <td bgcolor="pink">0.2</td>
  </tr>
  <tr>
    <td>Bonds</td>
    <td bgcolor="dodgerblue">0.2</td>
    <td bgcolor="dodgerblue">0.5</td>
    <td bgcolor="dodgerblue">1</td>
      <td bgcolor="pink">0.05</td>
  </tr>
</table></li>
        <br>
        <li>Recall the covariance of asset $i$ and asset $j$ equals the product of the correlation of asset $i$ and $j$ with the standard deviation of asset $i$ and the standard deviation of asset $j$.</li>
        <br>
        <li>After doing some computations, you compute the covariances corresponding to the different asset classes are as follows: </li>
        <br>
        <li><table>
  <tr>
    <th bgcolor="orange">Covariance</th>
    <th>Large Stocks</th>
    <th>Small Stocks</th>
    <th>Bonds</th>
  </tr>
  <tr>
    <td>Large Stocks</td>
    <td bgcolor="orange" >0.0144</td>
    <td bgcolor="orange">0.0144</td>
    <td bgcolor="orange">0.0012</td>
  </tr>
  <tr>
    <td>Small Stocks</td>
    <td bgcolor="orange">0.0144</td>
    <td bgcolor="orange">0.04</td>
    <td bgcolor="orange">0.005</td>
  </tr>
     <tr>
    <td>Bonds</td>
    <td bgcolor="orange">0.0012</td>
    <td bgcolor="orange">0.005</td>
    <td bgcolor="orange">0.0025</td>
  </tr>
</table></li>
        <br>
        <li>Given the above estimates, determine the asset allocation that produces the smallest amount of risk.</li>
        <br>
        <li>Specifically, find a portfolio comprised of these three asset classes whose return has the lowest variance.</li>
        <br>
        <li>Assume the portfolio can only hold long positions in each of the three asset classes and you are to be fully invested.</li>
    </ul>

In [2]:
# Similar to the problems above, we determine the decision variables for this problem
portfolio = cp.Variable(3, nonneg=True)
large_stocks = portfolio[0]
small_stocks = portfolio[1]
bonds = portfolio[2]

In [3]:
#We then create a covariance matrix using numpy array
covariance_matrix = np.array([[0.0144, 0.0144, 0.0012],
                              [0.0144, 0.04, 0.005],
                              [0.0012, 0.005, 0.0025]])

In [4]:
# Determine the objective of the problem which is to determine asset location that produces the smallest amount or "minimizes" risk.
objective = cp.Minimize(cp.quad_form(portfolio, covariance_matrix))

In [5]:
#Set the constraints
constraint_1 = (large_stocks + small_stocks + bonds == 1)
constraint_2 = (large_stocks >= 0)
constraint_3 = (small_stocks >= 0)
constraint_4 = (bonds >= 0)

In [6]:
problem = cp.Problem(objective, [constraint_1,
                              constraint_2,
                              constraint_3,
                              constraint_4])

In [7]:
print(problem)

minimize QuadForm(var1, [[0.0144 0.0144 0.0012]
 [0.0144 0.04   0.005 ]
 [0.0012 0.005  0.0025]])
subject to var1[0] + var1[1] + var1[2] == 1.0
           0.0 <= var1[0]
           0.0 <= var1[1]
           0.0 <= var1[2]


In [8]:
problem.solve()

0.0023834482758620687

In [9]:
problem.solve(verbose=True)

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Jan 16 06:27:53 PM: Your problem has 3 variables, 4 constraints, and 0 parameters.
(CVXPY) Jan 16 06:27:53 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 16 06:27:53 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 16 06:27:53 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 16 06:27:53 PM: Using cached ASA map, for faster compilation (bypassing reduction chain).
(CVXPY) Jan 16 06:27:53 PM: Finished problem compilation (took 

0.0023834482758620704

In [12]:
#Print the portfolio value and then round it
portfolio.value
np.round(portfolio.value, 4)

array([0.0897, 0.    , 0.9103])